# Import 


In [ ]:
import clr
clr.AddReference("RevitServices")
clr.AddReference('ProtoGeometry')
clr.AddReference("RevitAPI")
import Autodesk.Revit.DB
from Autodesk.Revit.DB import *
from Autodesk.DesignScript.Geometry import *
import RevitServices
from RevitServices.Persistence import DocumentManager
from RevitServices.Transactions import TransactionManager

# Creating list of thermal conductivities to be applied
Please, note that Thermal Resistance $R$=$\frac{h}{C}$, $R=[\frac{m^2 \times K}{W}]$\
where $C$ - thermal conductivity , $C=[\frac{W}{m \times K}]$ \
      $h$ - thickness of an element, fixed parameter, $h=[m]$

In [ ]:
FloorR=[1, 0.5, 0.25, 0.166667, 0.125, 0.1, 0.083333, 0.071429, 0.0625, 0.0555556, 0.05, 0.04]
WallsR=[0.6, 0.3, 0.15, 0.1, 0.075, 0.06, 0.05, 0.042857, 0.0375, 0.03333, 0.03, 0.024]
RoofR=[1, 0.5, 0.25, 0.166667, 0.125, 0.1, 0.083333, 0.071429, 0.0625, 0.0555556, 0.05, 0.04]

mat=FilteredElementCollector(doc).OfCategory(BuiltInCategory.OST_Materials)

materials=[]
for m in mat:
    materials.append(m.Name)

# Assigning required thermal assets to variables

In [ ]:
for m in mat:
    if m.Name=='Rigid insulation':
        Wall=doc.GetElement(m.ThermalAssetId).LookupParameter("Thermal Conductivity")
    if m.Name=='Wood Sheathing, Chipboard':
        Roof=doc.GetElement(m.ThermalAssetId).LookupParameter("Thermal Conductivity")
    if m.Name=='Structure, Timber Joist/Rafter Layer':
        Floor=doc.GetElement(m.ThermalAssetId).LookupParameter("Thermal Conductivity")

# Setting out gbXML export parameters

In [ ]:
opt=Analysis.EnergyAnalysisDetailModelOptions()
opt.EnergyModelType=Analysis.EnergyModelType.BuildingElement
opt.ExportMullions=False
opt.IncludeShadingSurfaces=False
opt.SimplifyCurtainSystems=True
opt.Tier=Analysis.EnergyAnalysisDetailModelTier.SecondLevelBoundaries

# Looping over all R values combinations and creaintg a model

In [ ]:
for i in range(len(FloorR)):
    t=Transaction(doc,"R change")
    t.Start()
    Floor.Set(FloorR[i]/0.3048)
    t.Commit()
    t.Dispose()
    for j in range(len(WallsR)):
        t=Transaction(doc,"R change")
        t.Start()
        Wall.Set(WallsR[j]/0.3048)
        t.Commit()
        t.Dispose()
        for k in range(len(RoofR)):
            t=Transaction(doc,"R change")
            t.Start()
            Roof.Set(RoofR[k]/0.3048)
            t.Commit()
            t.Dispose()
            c=Transaction(doc,"model creation")
            c.Start()
            model=Analysis.EnergyAnalysisDetailModel.Create(doc, opt)
            model.TransformModel()
            GBopt=GBXMLExportOptions()
            GBopt.ExportEnergyModelType=ExportEnergyModelType.BuildingElement
            doc.Export("C:\Users\Миша\Desktop\gbXMLs",str(round(0.5/FloorR[i],1))+","+str(round(0.3/WallsR[j],1))+","+str(round(0.5/RoofR[k],1)), GBopt)
            c.Commit()
            c.Dispose()

# Rotating the model, then repeating the previous step again

In [ ]:
base_point=doc.GetElement('9acbc50b-e118-424e-b6dd-04f4c76cb939-0001c1fb').LookupParameter("Angle to True North")
angle=Transaction(doc,"angle")
angle.Start()
base_point.Set(45)
angle.Commit()


Since I used export of a detailed model rather than conceptual, different glazing parameters could only be applied in Green Building Studio, one by one, by means of browser automatization 